# 导包
AutoModelForCausalLM: 用于加载模型

AutoTokenizer: 用于加载token分词器

TrainingArguments: 用于配置训练参数

Trainer: 用于训练模型

EarlyStoppingCallback: 用于提前结束训练 当评估损失不再下降时

In [ ]:

import json

import pandas as pd
import torch
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, Trainer, TrainingArguments, \
    EarlyStoppingCallback

from datasets import Dataset

# 数据加载

In [ ]:
train_data_path = './datasets/train_test/test0819.jsonl'
eval_data_path = './datasets/train_test/eval0819.jsonl'
model_path = r"D:\Pretrained_models\Qwen\Qwen2-1___5B-Instruct"
output_path = './Qwen2-1___5B-Instruct_ft_0819_1'

定义 load_json用于加载数据集 并使用view_data_distribution 查看数据集的标签分布

In [ ]:
def load_jsonl(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = [json.loads(line) for line in f]
        return pd.DataFrame(data)


def view_data_distribution(data_path, show_first=False):
    df = load_jsonl(data_path)
    print(f"total_count:{df.shape[0]}, true_count:{df['label'].sum()}, false_count:{(df['label'] == False).sum()}")
    print(json.dumps(df.iloc[1].to_dict(), indent=1, ensure_ascii=False)) if show_first else None


view_data_distribution(train_data_path, True)

In [ ]:
?json.dumps

数据序列化: 原始的训练数据是文本形式,而模型推理所需要的是数组,这中间需要用tokenizer进行文本到数字的序列化转换

1. 每个语言模型内部都维护了一个此表,里面维护了模型认识的所有词与数字编号的映射, 不同模型的词表是不一样的,我们需要使用基座模型所对应的此来创建tokenizer
2. Tokenizer是一个词元生成器,它首先通过分词算法将文本切分成独立的token列表, 再通过此表映射将每个token转换成语言模型可以处理的数字

``` text
Qwen2TokenizerFast(name_or_path='D:\Pretrained_models\Qwen\Qwen2-1___5B-Instruct', vocab_size=151643, model_max_length=32768, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)
```
上边这个tokenizer的输出信息表示: 词表中共有151643个词元,这个模型支持最大32kb的序列长度,并且还定义了开始标记<|im_start|>, 结束标记<|im_end|>, 填充标记<|endoftext|>, 这些特殊token需要在数据预处理时被正确的添加到文本中

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
tokenizer


`{'input_ids': [105043, 100165], 'attention_mask': [1, 1]}`

input_ids 是 你是谁 序列化后成token列表后的数字形式, 而attention_mask是一个与input_ids长度相同的数组, 用于指示模型应该关注那些token, 以及忽略哪些token, 填充token再模型推理时通常被忽略

注意:attention_mask的值通常为0或1, 1 表示该位置的token时有效输入(模型应该关注这个token), 0表示该位置的token是填充(模型在处理时应该忽略此token)

In [ ]:
tokenizer("你是谁")

定义输入文本的预处理函数,作用是按模型的输入要求将输入文本转为输入(input_ids), 掩码(attention_mask), 标签(labels) 三个序列

常见的疑问: 为什么input_ids 中不仅仅拼接了输入序列的token_id, 也拼接了输出label的token_id?

> Answer
    > - 训练阶段: 模型的目标是通过学习输入序列和目标标签序列之间的映射关系，从而使得模型能够根据输入生成正确的输出。这种映射关系是通过模型的参数（权重）来学习的，模型通过逐步调整这些权重来调整映射关系.
    > - 在推理阶段: 模型通过已经调整的映射关系来对输入进行推理,通过自回归来生成下一个token

In [ ]:
def preprocess(item, tokenizer, max_length=2048):
    system_message = "You are a helpful assistant"
    user_message = item['instruction'] + item['input']
    assistant_message = json.dumps({'is_fraud': item['label']}, ensure_ascii=False)

    input_ids, attention_mask, labels = [], [], []

    # 组装提示词并序列化
    instruction = tokenizer(
        f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n",
        add_special_tokens=False)

    # 响应序列化
    response = tokenizer(assistant_message, add_special_tokens=False)

    # 输入
    input_ids = instruction['input_ids'] + response['input_ids'] + [tokenizer.pad_token_id]

    # 注意力掩码
    attention_mask = instruction['attention_mask'] + response['attention_mask'] + [1]

    # -100 是一个特殊标记, 用于指示指令部分的token不应参与损失计算
    labels = [-100] * len(instruction['input_ids']) + response['input_ids'] + [tokenizer.pad_token_id]

    # 对输入长度做一个限制保护, 超出截断
    return {
        "input_ids": input_ids[:max_length],
        "attention_mask": attention_mask[:max_length],
        "labels": labels[:max_length]
    }

封装 一个load_dataset用于加载数据集,并使用preprocess函数来预处理所有的数据集

In [32]:
def load_dataset(train_path, eval_path, tokenizer):
    train_df = load_jsonl(train_path)
    train_ds = Dataset.from_pandas(train_df)
    train_dataset = train_ds.map(lambda x: preprocess(x, tokenizer), remove_columns=train_ds.column_names)

    eval_df = load_jsonl(eval_path)
    eval_ds = Dataset.from_pandas(eval_df)
    eval_dataset = eval_ds.map(lambda x: preprocess(x, tokenizer), remove_columns=eval_ds.column_names)

    return train_dataset, eval_dataset


train_dataset, eval_dataset = load_dataset(train_data_path, eval_data_path, tokenizer)
# print(train_dataset[0])
# print(eval_dataset[0])

tokenizer.decode(train_dataset[0]['input_ids'])

Map: 100%|██████████| 2348/2348 [00:00<00:00, 2658.86 examples/s]

{'input_ids': [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 151645, 198, 151644, 872, 271, 100431, 99639, 37474, 105051, 108704, 11, 220, 14880, 101042, 105051, 43815, 107189, 106037, 101052, 3837, 23031, 2236, 68805, 66017, 103929, 104317, 59151, 9623, 761, 97957, 25, 830, 91233, 8, 8997, 111586, 99445, 100201, 25, 49434, 239, 85106, 104857, 101118, 100158, 3837, 104163, 43288, 109228, 102509, 100635, 100709, 1773, 107809, 104169, 101883, 20450, 101037, 94432, 111955, 100339, 99425, 25, 84897, 60894, 73670, 3837, 35946, 101128, 112735, 20450, 110138, 103930, 1773, 100632, 14880, 105712, 3837, 97639, 104202, 114073, 105075, 104509, 100429, 3837, 105048, 104633, 29490, 102497, 97639, 1773, 151645, 198, 151644, 77091, 198, 4913, 285, 761, 97957, 788, 830, 92, 151643], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

'<|im_start|>system\nYou are a helpful assistant<|im_end|>\n<|im_start|>user\n\n下面是一段对话文本, 请分析对话内容是否有诈骗风险，以json格式输出你的判断结果(is_fraud: true/false)。\n受害者王伟: 我需要仔细考虑一下，毕竟这涉及到很大的资金投入。你能给我一些时间吗？\n骗子李红: 当然可以，我理解你需要时间来做决定。不过请记住，我们公司的名声是我们最重要的资本，你可以放心地信任我们。<|im_end|>\n<|im_start|>assistant\n{"is_fraud": true}<|endoftext|>'

# 模型准备
## step1 加载模型

可以先将模型加载进内存, 再使用model.to(device)将模型从内存移到指定的gpu设备上,这里用的模型比较小加上显卡资源有限,所以数据类型使用半精度的16位即可

In [34]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'


def load_model(model_path, device='cuda'):
    model = AutoModelForCausalLM.from_pretrained(model_path)
    # 开启梯度检查点时,要执行该方法
    model.enable_input_require_grads()
    return model.to(device)


model = load_model(model_path, device)
model

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear(in_features=1536, out_features=1536, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((1536,), eps=1e-06)
    (rotary_emb): Qw

## step2 插入微调参数
使用lora进行微调时,需要膝盖模型结构,这里将一个rank=8 的低秩矩阵插入到模型的每个decoder_layer中,在训练时只需要学习这个低秩矩阵,原模型的参数不改变
- target_modules: 定义了要对模型的哪些块做修改, 准确来说是具体哪些块中插入低秩矩阵
- r: 低秩矩阵的大小,值越小,模型学习的参数越少,这里使用默认的8
- lora_alpha: 一个缩放比例因子,控制着模型推理过程中将LoRA参数在模型整个参数中所占的比重大小,这里也按推荐配置为r的两倍
- lora_dropout: 训练过程中, 随机丢弃的神经元比例,目的是引入随机性来增强模型的泛化能力

In [36]:
def build_peft_model(model):
    config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],  #全部为线性层
        inference_mode=False,  #训练模型
        r=8,
        lora_alpha=16,
        lora_dropout=0.05
    )
    return get_peft_model(model, config)

peft_model = build_peft_model(model)
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

In [37]:
peft_model.print_trainable_parameters()

trainable params: 9,232,384 || all params: 1,552,946,688 || trainable%: 0.5945


## step3 构建训练器
配置训练参数,这块是需要重点关注和调整的地方,一些参数的理解如下:
- per_device_train_batch_size: 每个设备单次运算的小批量大小
- gradient_accumulation_steps: 梯度累积的步骤数, 原本是每4条数据更新一次参数, 加上梯度累计=4后相当于每16条数据更新一次参数,相当于变相增加batch_size大小
- num_train_epochs: 训练的总轮数, 默认值为3, 相当于所有的数据训练3遍
- eval_strategy: 评估策略, 可选有steps和epochs
- eval_steps: 训练多少评估一次模型的性能,每哥batchSize为1步,此参数在eval_strategy=steps时适用
- save_steps: 训练多少步自动保存一次模型参数
- learning_rate: 学习率
- load_best_model_at_end: 训练结束时自动加载最佳模型
- gradient_checkpoint: 是否启用梯度检查点,启用梯度检查点可以减少kvcache对内存的占用能节省内存

In [42]:
def build_train_arguments(output_path):
    return TrainingArguments(
        output_dir=output_path,
        per_device_train_batch_size=4, # 每个设备的训练批次大小
        gradient_accumulation_steps=4, # 梯度累计的步骤数,相当于增大批次大小
        logging_steps=10,
        num_train_epochs=3,
        evaluation_strategy="steps",
        eval_steps=10, # 设置评估的步数,与保存步数一致
        save_steps=100,
        learning_rate=1e-4,
        save_on_each_node=True,
        load_best_model_at_end=True, #在训练结束时加载最佳模型
        gradient_checkpointing=True # 启用梯度检查点以节省内存
    )

## step4创建训练器
重点理解以下的参数:
- eval_dataset: 评估数据集,设置了此参数,才会在训练过程中自动评估模型的性能,validation loss指标才会有值, 相当于边训练边验证效果
- data_collator: 控制如何将原始数据合并成批, DataCollatorForSeq2Seq会自动处理输入序列的填充, 使用tokenizer提供的填充标记, 将不同长度的序列填充到相同的长度,以避免在训练过程中因序列长度不同而产生错误
> 注: seq2seq模型中, 批量输入的多条文本数据通常具有不同的长度,而模型在进行矩阵运算时需要同一批次的数据有相同长度才能一起运算,否则会报错,所以需要指定padding=True参数来将输入序列填充到相同的长度
- EarlyStoppingCallback:用于设置提前结束训练的回调, early_stopping_patience=3 表示验证指标没有改进时, 经过3哥评估周期后提前停止训练
> 注:默认情况下, 训练会跑满train_dataset和num_train_epochs指定的所有数据集和训练轮次,但存在一些场景(比如过拟合)需要提前结束训练,此时就可以设置早停回调以免模型越训练越差,还有一个重要的点时避免浪费gpu算力成本

In [43]:
def build_trainer(model, tokenizer, args, train_dataset, eval_dataset):
    return Trainer(
        model=model,
        args=args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

## step5 开始训练

In [44]:
trainer = build_trainer(model, tokenizer, build_train_arguments(output_path), train_dataset, eval_dataset)
trainer.train()

D:\anaconda3\envs\extract-dialogue\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
10,0.420900,0.046621
20,0.049000,0.031533
30,0.037000,0.038272
40,0.032800,0.030056
50,0.034200,0.031802
60,0.024000,0.029236
70,0.031600,0.030744
80,0.029600,0.030548
90,0.027100,0.027959
100,0.029000,0.026669


Could not locate the best model at ./Qwen2-1___5B-Instruct_ft_0819_1\checkpoint-150\pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=180, training_loss=0.0500577946090036, metrics={'train_runtime': 4035.7518, 'train_samples_per_second': 1.746, 'train_steps_per_second': 0.109, 'total_flos': 5738068146456576.0, 'train_loss': 0.0500577946090036, 'epoch': 1.2244897959183674})